In [2]:
import torch

In [ ]:
# bdb2025.py
"""Common functions for loading and preparing Big Data Bowl 2025 data for modeling."""
from typing import Optional
from pathlib import Path
import pandas as pd
import numpy as np
import logging
logger = logging.getLogger(__name__)

def load_csv_data(
    paths: list[Path]
) -> list[pd.DataFrame]:
    # TODO: look into reading from template, i.e. tracking_week_*.csv
    return [pd.read_csv(path) for path in paths]


def load_tracking_data(
    weeks: Optional[int | list[int]]=None,
    db: Optional[Path]=None,
    folder: Optional[Path]=None,
) -> pd.DataFrame:
   
    MIN_WEEK = 1
    MAX_WEEK = 9

    if not weeks:
        # defualt to all data
        weeks = range(MIN_WEEK, MAX_WEEK)
    
    if isinstance(weeks, int):
        weeks = [weeks]
    weeks = set(weeks)
    
    for week in weeks:
        assert isinstance(week, int) and week >= MIN_WEEK and week <= MAX_WEEK, \
        f"provide integer weeks between {MIN_WEEK} and {MAX_WEEK}"

    logger.info(f"reading data from weeks: {weeks}")

    # TODO: Load from database
    if db and db.exists():
        return

    # load data from a folderectory
    if folder and folder.exists():
        logger.info(f"reading tracking from {folder}")
        paths = [Path(folder, f'tracking_week_{n}.csv') for n in weeks]

        dfs = load_csv_data(paths)
        logger.info("finshed reading tracking")
        return pd.concat(dfs)
    
    raise FileNotFoundError



def load_player_data(
    db: Optional[Path]=None,
    folder: Optional[Path]=None,
) -> pd.DataFrame:
    
    if db and db.exists():
        return

    if folder and folder.exists():
        logger.info(f"reading player data from {folder}")
        path = Path(folder, 'players.csv')

        return pd.read_csv(path)
    

def load_play_data(
        db: Optional[Path]=None,
        folder: Optional[Path]=None,
) -> pd.DataFrame:
    
    if db and db.exists():
        return

    if folder and folder.exists():
        logger.info(f"reading play data from {folder}")
        path = Path(folder, 'plays.csv')

    return pd.read_csv(path)


def clean_player_data(player_df: pd.DataFrame) -> pd.DataFrame:
    logger.info("cleaning player data")
    
    # clean height
    feet = pd.to_numeric(player_df['height'].str.split('-', expand=True)[0])
    inches = pd.to_numeric(player_df['height'].str.split('-', expand=True)[1])
    player_df['height_inches'] = 12 * feet + inches

    # create zscore for height
    player_df['height_z'] = (player_df['height_inches'] - player_df['height_inches'].mean()) / player_df['height_inches'].std()
    player_df['weight_z'] = (player_df['weight'] - player_df['weight'].mean()) / player_df['weight'].std()

    logger.info("finished cleaning player data")
    return player_df


def clean_tracking_data(tracking_df: pd.DataFrame) -> pd.DataFrame:
    logger.info("cleaning tracking data")

    # angles to radians
    tracking_df['o'] = ((-1 * tracking_df['o'] + 90) % 360) * np.pi / 180
    tracking_df['dir'] = ((-1 * tracking_df['dir'] + 90) % 360) * np.pi / 180

    # standardize locations to the ball snap location
    ball = (
        tracking_df
        .loc[
            (tracking_df['event'] == 'ball_snap') & 
            (tracking_df['club'] == 'football'),
             
            ['gameId', 'playId', 'frameId', 'x', 'y']
        ]
    )
    tracking_df = tracking_df.merge(
        ball, # ball info for starting time
        how='left', 
        on=('gameId', 'playId'),
        suffixes=('', '_ball')
    )

    tracking_df['x'] = tracking_df['x'] - tracking_df['x_ball']
    tracking_df['y'] = tracking_df['y'] - tracking_df['y_ball']

    # normalize play directions to the right
    tracking_df['x'] = ((tracking_df['playDirection'] == 'left') * -2 + 1) * tracking_df['x']
    tracking_df['o'] = ( ((tracking_df['playDirection'] == 'left') * np.pi) + tracking_df['o'] ) % (2 * np.pi) 
    tracking_df['dir'] = ( ((tracking_df['playDirection'] == 'left') * np.pi) + tracking_df['dir'] ) % (2 * np.pi) 

    # x and y components of orientation and velocity
    tracking_df['ox'] = np.cos(tracking_df['o'])
    tracking_df['oy'] = np.sin(tracking_df['o'])

    tracking_df['vx'] = np.cos(tracking_df['dir']) * tracking_df['s']
    tracking_df['vy'] = np.sin(tracking_df['dir']) * tracking_df['s']

    tracking_df['stop_point'] = pd.NA
    tracking_df.loc[tracking_df['event'].isin(['pass_outcome_incomplete', 'qb_sack', 'tackle']), 'stop_point'] = True 
    # fill stopping points forward to filter out moments after a defined route stop
    tracking_df['stop_point'] = tracking_df.groupby(['gameId', 'playId', 'nflId'])['stop_point'].ffill()
    
    tracking_df = tracking_df.loc[pd.isna(tracking_df['stop_point'])].copy()

    # filter
    tracking_df = tracking_df.loc[
        (tracking_df['frameId'] >= tracking_df['frameId_ball']) & # after the starting point
        (tracking_df['club'] != 'football')
        
        #['gameId', 'playId', 'frameId', 'nflId', 'x', 'y', 'vx', 'vy', 'a', 'ox', 'oy']
    ].copy()
    
    logger.info("finished cleaning tracking data")
    return tracking_df


def mirror_tracking_plays(tracking_df: pd.DataFrame) -> pd.DataFrame:
    logger.info("mirroring tracking data")
    mirrored_df = tracking_df.copy()

    mirrored_df['y'] = mirrored_df['y'] * -1
    mirrored_df['vy'] = mirrored_df['vy'] * -1
    mirrored_df['oy'] = mirrored_df['oy'] * -1

    mirrored_df['mirrored'] = True
    tracking_df['mirrored'] = False

    logger.info("finished mirroring tracking data")

    return pd.concat([tracking_df, mirrored_df])


def prepare_static_data(tracking_df: pd.DataFrame, play_df: pd.DataFrame, player_df: pd.DataFrame ) -> pd.DataFrame:
    logger.info("preparing static data")
    # join 

    joined_df = (
        tracking_df
        .groupby(['gameId', 'playId', 'nflId', 'mirrored'], as_index=False)
        .first()
        .merge(
            play_df,
            how='left',
            on=('gameId', 'playId')
        )
        .merge( # player info for positions
            player_df, 
            how='left',
            on='nflId'
        )
    )
    joined_df['offense'] = joined_df['club'] == joined_df['possessionTeam']

    # filter
    joined_df = joined_df[        
        ['gameId', 'playId', 'nflId', 'mirrored', 'height_z', 'weight_z', 'position', 'offense']
    ].copy()

    joined_df = pd.get_dummies(joined_df)
    logger.info("finished preparing static data")

    return joined_df

In [ ]:
# datasets.py
"""Functions for perparing datasets for modeling"""
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline
from torch.utils.data import Dataset


def interpolate_movement(tracking_df: pd.DataFrame) -> np.ndarray:

    play_index = tracking_df.set_index(['gameId', 'playId', 'mirrored']).sort_index()

    num_obs = 40
    interpolated_columns = ['x', 'y', 'ox', 'oy', 'vx', 'vy', 'a']
    ids = play_index.index.unique()
    num_plays = len(ids)
    routes_per_play = tracking_df.groupby(['gameId', 'playId'])['nflId'].nunique(dropna=True).max()

    logger.info(f"interpolating data {len(interpolated_columns)} of data")
    # array for interpolated movement
    movement_arr = np.zeros( (num_plays, routes_per_play, num_obs, len(interpolated_columns)), dtype=np.float32)

    for i, (gameid, playid, mirrored) in enumerate(ids): 
        
        play: pd.DataFrame = play_index.loc[gameid, playid, mirrored]
        
        for ii, nflid in enumerate(play['nflId'].dropna().unique()):

            player: pd.DataFrame = play.loc[play['nflId'] == nflid]
            min_frame = player['frameId'].min()
            frame_range = player['frameId'].max() - min_frame
            t = np.array((player['frameId'] - min_frame) / frame_range)

            player_data = np.array(player[interpolated_columns])

            # smooth the info about the player over time
            spline = CubicSpline(t, player_data, axis=0, extrapolate=False)

            # take n observations
            observation_times = np.arange(num_obs) / num_obs

            # populate an array with smoothed values
            movement_arr[i, ii, ...] = spline(observation_times)

    logger.info("finished interpolating data")

    return movement_arr


def perpare_data_arrays(static_df: pd.DataFrame, movement_arr: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    logger.info("preparing arrays")
    play_index = static_df.set_index(['gameId', 'playId', 'mirrored'])
    
    num_plays = len(play_index.index.unique())
    routes_per_play = static_df.groupby(['gameId', 'playId', 'mirrored'])['nflId'].nunique(dropna=True).iloc[0]

    assert (static_df.groupby(['gameId', 'playId', 'mirrored'])['nflId'].nunique(dropna=False) == routes_per_play).all(), \
    "Different values across player dimension"
    
    num_columns = len(play_index.columns) - 1 # subtract the nflId column
    player_info = np.zeros( (num_plays, routes_per_play, num_columns), dtype=np.float32) 

    for i, (name, group) in enumerate(play_index.groupby(['gameId', 'playId', 'mirrored'])):
        player_info[i, ...] = group.drop(columns=['nflId'])

    input_arr = np.concat(
        [
            movement_arr.reshape((num_plays, routes_per_play, -1)), # combine final axes 
            player_info, # concat fixed info
        ],
        axis=-1
    )

    target_arr = movement_arr[..., :2].reshape((num_plays, routes_per_play, -1))

    logger.info("finished preparing arrays")
    return input_arr, target_arr


def mask_input(input_arr: np.ndarray, n: int) -> tuple[np.ndarray, np.ndarray]:
    logger.info("masking input")
    mask_pct = .15

    # "mask token"
    mask = np.zeros(n)

    n_player_vectors = np.prod(input_arr.shape[:-1])
    mask_idx = np.random.choice(
        n_player_vectors,
        size=int(n_player_vectors * mask_pct),
        replace=False
    ) # random choice over the product of the first axes

    # determine vectors to mask

    masked_players_arr = input_arr.reshape((n_player_vectors, -1)).copy()
    masked_players_arr[mask_idx, :n] = mask # TODO: test with 0 for readibility
    mask_arr = np.zeros(masked_players_arr.shape[0], dtype=np.float32)
    mask_arr[mask_idx] = 1

    logger.info("finished masking input")

    return masked_players_arr.reshape(input_arr.shape), mask_arr.reshape(input_arr.shape[:2])


def train_val_test_split(n: int):

    idx = np.arange(n)

    cats = np.random.choice(a=[0, 1, 2], size=n, replace=True, p=[.7, .15, .15]) # 70, 15, 15

    split = np.stack([idx, cats], axis=-1)

    train_idx = split[split[:, -1] == 0, 0]
    val_idx = split[split[:, -1] == 1, 0]
    test_idx = split[split[:, -1] == 2, 0]

    return train_idx, val_idx, test_idx
    

class NflDataset(Dataset):
    
    def __init__(self, input_arr: np.ndarray, target_arr: np.ndarray, mask_arr: np.ndarray, idx: np.ndarray, device: torch.DeviceObjType=torch.device("cpu")):

        # make sure there's the right amount of input observatinos and target observations
        assert input_arr.shape[0] == target_arr.shape[0], "# Observations mismatch between input and target"

        self.input_t: torch.Tensor = torch.Tensor(input_arr[idx].copy()).to(device)
        self.target_t: torch.Tensor = torch.Tensor(target_arr[idx].copy()).to(device)
        self.mask_t: torch.Tensor = torch.Tensor(mask_arr[idx].copy()).to(device)

        return
    
    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        
        return self.input_t[idx], self.target_t[idx], self.mask_t[idx].unsqueeze(-1)


    def __len__(self) -> int:

        return self.input_t.shape[0]
    
    


In [ ]:
# model.py
"""Model Architecture"""
import torch
import numpy as np

class NflBERT(torch.nn.Module):
    """Bert style model for encoding nfl player movement
    
    BERT uses a nearly identical transformer encoder as "Attention is all you need" and thus
    similar to the provided torch TransformerEncoder layer"""

    def __init__(
        self,
        feature_dim: int,
        output_dim: int,
        hidden_size: int=512,
        num_layers: int=8,
        num_heads: int=8,
        ffn_size: int=2048,
        ffn_act: str="gelu",
        dropout: float=.3
    ):
        super().__init__()
        self.norm_layer = torch.nn.BatchNorm1d(feature_dim)
        
        self.embed = torch.nn.Sequential(
            torch.nn.Linear(feature_dim, hidden_size),
            torch.nn.GELU(),
            torch.nn.LayerNorm(hidden_size),
            torch.nn.Dropout(dropout)
        )

        self.transformer = torch.nn.TransformerEncoder(
            torch.nn.TransformerEncoderLayer(
                d_model=hidden_size,
                nhead=num_heads,
                dim_feedforward=ffn_size,
                dropout=dropout,
                activation=ffn_act,
                batch_first=True
            ),
            num_layers=num_layers
        )

        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, hidden_size // 4),
            torch.nn.GELU(),
            torch.nn.Dropout(dropout),
            torch.nn.LayerNorm(hidden_size // 4),
            torch.nn.Linear(hidden_size // 4, output_dim)
        )


    
    def forward(self, x: torch.Tensor):
        
        # x: [B: batch_size, P: # of players, F: feature_len]
        B, P, F = x.size()

        # Normalize features
        x = self.norm_layer(x.permute(0, 2, 1)).permute(0, 2, 1)  # [B,P,F] -> [B,P,F]

        # Embed features
        x = self.embed(x)  # [B,P,F] -> [B,P,M: model_dim]

        # Apply transformer encoder
        x = self.transformer(x)  # [B,P,M] -> [B,P,M]

        # Decode to predict tackle location
        x = self.decoder(x)  # [B, P, M] -> [B,P,O]

        return x



In [ ]:
# load data
logger.setLevel(logging.INFO)

bdb_folder = Path("/Users/henrykraessig/code/bdb2025/data/")

tracking = pd.read_csv("minimal_tracking.csv")
tracking = load_tracking_data(folder=bdb_folder)
players = load_player_data(folder=bdb_folder)
plays = load_play_data(folder=bdb_folder)

players = clean_player_data(players)

tracking = clean_tracking_data(tracking)
tracking = mirror_tracking_plays(tracking)

static = prepare_static_data(tracking, plays, players)
num_static_cols = len([col for col in static.columns if 'Id' not in col and 'mirrored' not in col])

In [ ]:
movement = interpolate_movement(tracking)
input, target= perpare_data_arrays(static, movement)
input, mask = mask_input(input, np.prod(movement.shape[2:]))

train_idx, val_idx, test_idx = train_val_test_split(input.shape[0])


train_dataset = NflDataset(input, target, mask, train_idx)
val_dataset = NflDataset(input, target, mask, val_idx)

In [ ]:
# data eval & plotting to debug
import plotly.express as px

def plot_anim(inp: np.ndarray, tar: np.ndarray, idx: int, static_cols: int, interp: int):
    
    # inp: [x, 22, 303]

    play = inp[idx, :, :-static_cols].reshape((inp.shape[1], interp, -1)) # 22, 40, 7
    play = play[..., :2] # 22, 40, 2

    ids = np.repeat(range(play.shape[0]), play.shape[1]).reshape(list(play.shape[:-1]) + [1]) # 22, 40, 1
    frames = np.tile(range(play.shape[1]), play.shape[0]).reshape(list(play.shape[:-1]) + [1])


    # target
    tar_filter = ~play.all(axis=(-1, -2))
    target = tar[idx, tar_filter] # ?, 80
    
    target = target.reshape((-1, interp, 2))

    play[tar_filter, ...] = target
    ids[tar_filter, ...] = -1

    play_id = np.concat([play, ids, frames], axis=-1)
    

    df = pd.DataFrame(play_id.reshape((-1, play_id.shape[-1])), columns=['x', 'y', 'id', 'frame'])
    df['target'] = (df['id'] == -1)

    fig = px.scatter(
        df,
        x="x",
        y="y",
        color='target',
        animation_frame="frame",
        title=f"test"
    )


    return fig

plot_anim(input.copy(), target.copy(), 0, num_static_cols, 40)

In [ ]:
import plotly.graph_objects as go

def plot_eval(tar: np.ndarray, pred: np.ndarray, mask: np.ndarray, idx: int, interp: int):
    
    if mask.shape[-1] == 1:
        mask = mask.squeeze(axis=-1)

    # static (not animated) plotly plot to visualize model predictions against groundtruth

    # plot the first of the entire input array
    targets = tar.reshape(*tar.shape[:-1], interp, -1)
    predictions = pred.reshape(*pred.shape[:-1], interp, -1)

    starts = targets[idx, :, 0, :2] # first frame of x&y for 22 players

    # get the masked player's movements
    gt_movement = targets[idx, mask[idx].astype(np.bool)]

    # get predicted movements
    pred_movement = predictions[idx, mask[idx].astype(np.bool)]


    # plot start locations and lines for gt and pred movement
    fig = go.Figure()

    fig.add_trace( # start locations
        go.Scatter(
            y = starts[..., 0],
            x = starts[..., 1],
            mode = 'markers', 
            showlegend=False
        )
    )

    for i, (player_gt, player_pred) in enumerate(zip(gt_movement, pred_movement)):

        fig.add_trace(
            go.Scatter(
                y = player_gt[..., 0],
                x = player_gt[..., 1],
                name = f"groundtruth {i}",
                showlegend = False,
                mode = 'lines',
                marker = dict(
                    color = 'black'
                )
            )
        )

        fig.add_trace(
            go.Scatter(
                y = player_pred[..., 0],
                x = player_pred[..., 1],
                name = f"predicted {i}",
                showlegend = False,
                mode = 'lines',
                marker = dict(
                    color = 'red'
                )
            )
        )
    
    return fig


In [ ]:
hparams = {
    "batch_size": 32,
    "hidden_size": 64,
    "num_layers": 2,
    "num_heads": 2,
    "ffn_size": 64 * 4,
    "epochs": 100,
    "lr": 0.001,
    "patience": 10,
    "device": "cpu",
}

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, hparams["batch_size"], True)
val_loader = torch.utils.data.DataLoader(train_dataset, hparams["batch_size"], False)
model = NflBERT(
    input.shape[-1], # in 
    target.shape[-1], # out
    hidden_size=hparams["hidden_size"],
    num_layers=hparams["num_layers"], 
    num_heads=hparams["num_heads"], 
    ffn_size=hparams["ffn_size"]
).to(hparams["device"])

loss_fn = torch.nn.SmoothL1Loss()
optim = torch.optim.AdamW(model.parameters(), lr=hparams["lr"])

In [ ]:
# prepare the loss function
# create the optimizer
# create plotting utils
# analyze the play normailzation - vanishing gradients?
# fix masking to only loss over masked samples & limit max masked samples per play

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from tqdm.notebook import tqdm
import io

writer = SummaryWriter('./runs/allweeks')

In [ ]:
def check_early_stopping(loss: float, best_val_loss: float, no_improvement: int, patience: int) -> tuple[bool, float, int]:
    if loss < best_val_loss:
        return True, loss, 0 # reset no improvement
    else:
        no_improvement += 1
        if no_improvement >= patience:
            return False, best_val_loss, patience
        else:
            return True, best_val_loss, no_improvement
        

for epoch in tqdm(range(hparams["epochs"])):

    c = 0
    train_loss = 0
    val_loss = 0
    best_val_loss = float("inf")
    no_improvement = 0

    model.train()
    for (inp, tar, mask) in train_loader:

        optim.zero_grad()
        x = model(inp)

        loss = loss_fn(mask*x, mask*tar)
        train_loss += loss
        c += 1
        loss.backward()
        
        optim.step()
    # log average loss
    writer.add_scalar('Loss/train', train_loss / c, epoch)

    c = 0
    model.eval()
    with torch.no_grad():
        for (inp, tar, mask) in val_loader:
            x = model(inp)

            loss = loss_fn(x*mask, tar*mask)
            val_loss += loss
            c += 1

    # log average loss
        writer.add_scalar('Loss/val', val_loss / c, epoch)

        cont, best_val_loss, no_improvement = check_early_stopping(val_loss, best_val_loss, no_improvement, hparams["patience"])
        if not cont: # early stopped
            print(f"Early stopping at epoch {epoch}\nBest Loss: {best_val_loss}")
            break

        if epoch % 100 == 0:
            # log a sample play
            pred = model(inp)
            
            fig = plot_eval(tar.numpy().copy(), pred.numpy().copy(), mask.numpy().copy(), 0, 40)
            img_bytes = fig.to_image(format='jpg')
            img = Image.open(io.BytesIO(img_bytes))
            
            writer.add_image(f"sample/{epoch}", np.array(img).transpose([2, 0, 1]))

            print(f"Epoch: {epoch}\tVal Loss: {val_loss/c:.3f}")
        
writer.flush()
writer.close()